In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import gc
from IPython.core.display import display

def getHeatMap(newdf,metdf,dirpath,month):
    
    #Creazione dataframe finale
    
    finaldf=pd.DataFrame(columns=['hours','bool','power_dev','power_met'])
    
    finaldf['hours']=newdf['hours']
    finaldf['bool'] = np.where(((newdf.power <= metdf.power) | ((np.isnan(newdf.power)) & (metdf.power >= 0))),1,0).astype(int)
    finaldf['power_met']=metdf['power']
    finaldf['power_dev']=newdf['power']
    writedf=finaldf[finaldf['bool']==0]
    '''
    heatDir=os.path.join(os.path.abspath('..'),os.path.join("CHECK_FINAL","EB"+dirpath[17]))
    if not os.path.exists(heatDir):
        os.makedirs(heatDir)
    heatmapPath=os.path.join(heatDir,month+'.csv')
    print(heatmapPath)
    writedf.to_csv(heatmapPath,index=False)
    '''
    metdf.at[metdf.hours.isin(writedf.hours),'power'] = finaldf['power_dev']
    metdf = metdf[['hours', 'power']]
    
    metDir=os.path.join(os.path.abspath('..'),os.path.join("METER","EB"+dirpath[17]))
    if not os.path.exists(metDir):
        os.makedirs(metDir)
    metPath=os.path.join(metDir,month+'.csv')
    metdf.to_csv(metPath,index=False)

    print('HO FINITO')
   
 


    

    
#Funzione che ritorna dataframe in base al mese   
def switch_demo(boolval):
    global jandf
    global metjandf
    if(boolval==False):
        return jandf
    else:
        return metjandf
#Funzione che assegna le modifiche al dataframe in base al mese
def switch_df(df,boolval):
    global jandf
    global metjandf
    if(boolval==False):
        jandf=df
    else:
        metjandf=df
def process(df,filepath,count,boolval):
    global jandf
    global metjandf
    
    df['timestamp'] = pd.to_datetime(df['timestamp'],utc=True).dt.tz_convert('Europe/Rome')
    
    #Aggiungi una colonna contenente  data e ora per heatmap
    df['hours']=df['timestamp']#.dt.strftime("%d-%b-%Y (%H-%M-%S)")
    
    df=df.set_index('timestamp')
    
   

    filterdf=switch_demo(boolval)
   
    df = df.loc[df['power'].notnull()]
    if boolval == False:
        filterdf.loc[(filterdf.hours.isin(df.hours)) & (filterdf['reset'] == 0), 'power'] = 0
        filterdf.loc[filterdf.hours.isin(df.hours) , 'power'] = filterdf['power']+df['power']
        filterdf.loc[filterdf.hours.isin(df.hours) , 'reset'] = 1

    else:
        filterdf.loc[filterdf.hours.isin(df.hours) , 'power'] = df['power']

        
    switch_df(filterdf,boolval)
    
     

In [2]:
#DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019","\Dati puntuali EB3 Gennaio-Giugno 2019","\Dati puntuali EB4 Gennaio-Giugno 2019","\Dati puntuali EB5 Gennaio-Giugno 2019","\Dati puntuali EB6 Gennaio-Giugno 2019","\Dati puntuali EB7 Gennaio-Giugno 2019","\Dati puntuali EB8 Gennaio-Giugno 2019","\Dati puntuali EB9 Gennaio-Giugno 2019"]
DIRPATH=["/Dati puntuali EB1 Gennaio-Giugno 2019"]
count=0
for dirpath in DIRPATH:
    count=count+1
    print("Start "+str(count))
    dirp=os.path.join(os.path.abspath('..'),"data_rec"+dirpath+"/")
    filelist=[f for f in os.listdir(dirp)]
    jandf = metjandf= pd.DataFrame({
        'date': pd.date_range(
            start=pd.Timestamp('2019-04'),
            end=pd.Timestamp('2019-04') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
            freq='S',  # <--- try '3h', '6h', '12h' if you want
            closed='left',
            tz='Europe/Rome'
                )
        })
    pd.set_option('display.max_rows', 10000)
    #display(jandf)
    jandf['hours'] = jandf['date']
    metjandf['hours'] = metjandf['date']
    jandf = jandf.set_index('date')
    metjandf = metjandf.set_index('date')
    jandf['reset'] = metjandf['reset'] = 0
    #jandf['hours']=jandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")
    
    #metjandf['hours']=metjandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")

    jandf['power']=metjandf['power'] = np.nan
    
    for filepath in filelist:
        dataPath=os.path.join(dirp,filepath)
        print(filepath)
        if("Meter" in filepath):
            boolval=True
        else:
            boolval=False
        for chunck in pd.read_csv(dataPath, sep=',',chunksize=500000):
            process(chunck,filepath,count,boolval)
            print('Fine')
            print('-----')
    
    #Creazione heatmap
    getHeatMap(jandf,metjandf,dirpath,'Apr') 
    
    
        




Start 1
Dati puntuali Frigo EB1 2019.csv
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Dati puntuali Lavastoviglie EB1 2019.csv
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Dati puntuali Lavatrice EB1 2019.csv
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Dati puntuali Meter EB1 2019.csv
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
Fine
-----
HO FINITO
